<a href="https://colab.research.google.com/github/hbthekraken/Job_Shift_AssignmentProject/blob/main/Job_Shift_Assigner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this code, there is a function which can build concrete Pyomo models. These models can assign jobs that are sequentially processed over a series of processors to the shifts, so they model the planning. These models can also support parallelism for machines. Each job represents a fixed quantity (lot) of parts of the same type that can be done in a shift.

Due dates constraints, sequentiality constraints, parallelism constraints etc. are therefore available. With these, the model can be tested (and used) by generating a hypothetical list of parts to be made and then converting them into jobs. Conversion is generally needed because each job must represent what can be done in one shift but more parts might be needed than that.

In [ ]:
import pyomo.environ as pyo
import highspy as hp
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt


plt.style.use("ggplot")

Calling necessary libraries for modelling and testing.

In [ ]:
slvr = "appsi_highs"
print(pyo.SolverFactory(slvr).available())

True


Here, we check if the HiGHS solver is available. If it is not, another available solver must be supplied through the variable ```slvr```.




In [ ]:
def build_concr_model(N, K, L, J_NL, alpha_coefficients, d_dates, p_times, sigma_availabilities):
  model_ = pyo.ConcreteModel()

  # Some model parameters related to index sets
  model_.N = pyo.Param(within=pyo.PositiveIntegers, initialize=N)
  model_.K = pyo.Param(within=pyo.PositiveIntegers, initialize=K)
  model_.L = pyo.Param(within=pyo.PositiveIntegers, initialize=L)

  # model index sets here, J_NL being the  subset of J
  model_.J = pyo.RangeSet(1, model_.N)
  model_.S = pyo.RangeSet(1, model_.K)
  model_.M = pyo.RangeSet(1, model_.L)

  model_.J_NL = pyo.Set(within=model_.J, initialize=J_NL)

  # Decision Variables which are all binary
  model_.x = pyo.Var(model_.S, model_.J, model_.M, within=pyo.Binary)

  # Actual model parameters related to the problem
  model_.alpha = pyo.Param(model_.J, within=pyo.PositiveReals, initialize=alpha_coefficients) # Importance Factors - coefficients
  model_.d = pyo.Param(model_.J, within=pyo.PositiveReals, initialize=d_dates) # Due dates for Not-Late jobs, although indexed for all j in J, only the ones for the j in J_NL will be used
  model_.p = pyo.Param(model_.J, model_.M, within=pyo.Binary, initialize=p_times) # Processing time for job j on processor/machine on m
  model_.sigma = pyo.Param(model_.S, model_.M, within=pyo.NonNegativeReals, initialize=sigma_availabilities) # Availability (can be > 1 if parallel machines exist) of processor-m on shift-s

  # Model Objective
  def z(_model):
    return sum(_model.alpha[_j] * sum(_s * _model.x[_s, _j, _model.L] for _s in _model.S) for _j in _model.J)

  model_.objective = pyo.Objective(rule=z, sense=pyo.minimize)

  # Constraints:
  # Enforcing due dates:
  def EnforceDueDates(_model, _j_NL):
    return sum(_s * _model.x[_s, _j_NL, _model.L] for _s in _model.S) <= _model.d[_j_NL]

  # Machining time constraint
  def MachineTimeConstraint(_model, _s, _m):
    return sum(_model.p[_j, _m] * _model.x[_s, _j, _m] for _j in _model.J) <= _model.sigma[_s, _m]

  # One Shift for One process Constraint
  def OneShiftConstraint(_model, _j, _m):
    return sum(_model.x[_s, _j, _m] for _s in _model.S) == 1

  # Sequantieal Machining Constraint
  def SequentialityConstraint(_model, _j, _m):
    if _m < _model.L:
      return _model.p[_j, _m] + sum(_s * (_model.x[_s, _j, _m] - _model.x[_s, _j, _m + 1]) for _s in _model.S) <= 0.0
    else:
      return pyo.Constraint.Skip

  model_.EnforceDueDates = pyo.Constraint(model_.J_NL, rule=EnforceDueDates)
  model_.MachineTimeLimit = pyo.Constraint(model_.S, model_.M, rule=MachineTimeConstraint)
  model_.OneShiftRule = pyo.Constraint(model_.J, model_.M, rule=OneShiftConstraint)
  model_.SequentialityConstraint = pyo.Constraint(model_.J, model_.M, rule=SequentialityConstraint)
  return model_

This is the function that takes the data (parameters of the model) as input and outputs a concrete Pyomo model. The details of the

In [ ]:
job_number = 5
shift_number = 10
machine_seq_length = 4
alpha_data = np.random.uniform(10.0, 100.0, size=job_number)
d_data = 5 + 0 * np.ones(job_number, dtype=int)
p_data = np.ones((job_number, machine_seq_length,), dtype=int)
sigma_data = 2 * np.ones((shift_number, machine_seq_length,), dtype=int)

auto_built_concr_model = build_concr_model(job_number, shift_number, machine_seq_length,
                                           tuple([2]),
                                           {i + 1 : alpha_data[i] for i in range(alpha_data.shape[0])},
                                           {i + 1 : d_data[i] for i in range(d_data.shape[0])},
                                           {(i + 1, j + 1) : p_data[i, j] for i in range(p_data.shape[0]) for j in range(p_data.shape[1])},
                                           {(i + 1, j + 1) : sigma_data[i, j] for i in range(sigma_data.shape[0]) for j in range(sigma_data.shape[1])}
                                           )

In [ ]:
opt = pyo.SolverFactory(slvr)
results = opt.solve(auto_built_concr_model, tee=True)
x_opt = auto_built_concr_model.x.extract_values()

Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms
MIP  has 76 rows; 200 cols; 710 nonzeros; 200 integer variables (200 binary)
Coefficient ranges:
  Matrix [1e+00, 1e+01]
  Cost   [3e+01, 9e+02]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 5e+00]
Presolving model
76 rows, 195 cols, 690 nonzeros  0s
75 rows, 137 cols, 474 nonzeros  0s
64 rows, 116 cols, 382 nonzeros  0s
59 rows, 116 cols, 382 nonzeros  0s

Solving MIP model with:
   59 rows
   116 cols (116 binary, 0 integer, 0 implied int., 0 continuous, 0 domain fixed)
   382 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; J => Feasibility jump;
     H => Heuristic; L => Sub-MIP; P => Empty MIP; R => Randomized rounding; Z => ZI Round;
     I => Shifting; S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bou

In [ ]:
for _j in range(1,6):
  for _m in range(1,5):
    print(f"The job-{_j} on machine-{_m} is done at shift-{sum([_s * int(x_opt[(_s, _j, _m)]) for _s in range(1,11)])}. It had alpha of {alpha_data[_j - 1]:.3f}")

The job-1 on machine-1 is done at shift-1. It had alpha of 90.511
The job-1 on machine-2 is done at shift-2. It had alpha of 90.511
The job-1 on machine-3 is done at shift-3. It had alpha of 90.511
The job-1 on machine-4 is done at shift-4. It had alpha of 90.511
The job-2 on machine-1 is done at shift-1. It had alpha of 35.609
The job-2 on machine-2 is done at shift-2. It had alpha of 35.609
The job-2 on machine-3 is done at shift-3. It had alpha of 35.609
The job-2 on machine-4 is done at shift-4. It had alpha of 35.609
The job-3 on machine-1 is done at shift-2. It had alpha of 28.252
The job-3 on machine-2 is done at shift-3. It had alpha of 28.252
The job-3 on machine-3 is done at shift-4. It had alpha of 28.252
The job-3 on machine-4 is done at shift-5. It had alpha of 28.252
The job-4 on machine-1 is done at shift-2. It had alpha of 29.260
The job-4 on machine-2 is done at shift-3. It had alpha of 29.260
The job-4 on machine-3 is done at shift-4. It had alpha of 29.260
The job-4 

Write a new part generator.

Adapt a new part-to-job converter.

Test it with the model builder and solve for the system.

Visaulize the output of the solver.

In [ ]:
def prod_to_jobs(prod_df):
  jobs = {}
  N = len(prod_df.index)
  job_count = 0
  part_count = 0
  while N > 0:
    name = prod_df.index[part_count]
    a_part = prod_df.iloc[part_count]
    Qty, LS, DT, TfL, alpha = tuple(a_part)
    Qty, LS = int(Qty), int(LS)
    if Qty % LS == 0:
      k = Qty // LS
      for i in range(k):
        jobs[job_count] = (str(name) + f'_{i}', int(LS), int(TfL), int(DT), alpha, job_count) # for a job, in the strict technical sense,
        # processing time, due time, importance factor, and its assigned shift are required to be defined
        # name and part count are present for convenience
        job_count += 1
    elif Qty < LS:
      if Qty / LS >= 0.5 and alpha > 1.5:
        jobs[job_count] = (str(name), int(LS), int(TfL), int(DT), alpha, job_count)
      else:
        pass
    elif Qty > LS:
      if alpha >= 1.0:
        k = 1 + (Qty // LS)
        for i in range(k):
          jobs[job_count] = (str(name) + f'_{i}', int(LS), int(TfL), int(DT), alpha, job_count)
          job_count += 1
      else:
        k = Qty // LS
        for i in range(k):
          jobs[job_count] = (str(name) + f'_{i}', int(LS), int(TfL), int(DT), alpha, job_count)
          job_count += 1
    else:
      raise Exception("Unknown error.")

    # print(f'Current job count={job_count}, with part_count={part_count}, N={N}, and part name={name}')
    N -= 1
    part_count += 1
  return jobs

In [ ]:
gen = np.random.default_rng()
test_data = np.array([
    [gen.integers(low=1, high=19), gen.integers(low=1, high=16), gen.integers(low=1, high=26), gen.integers(low=20, high=201), gen.uniform(0.1, 5.0)] for _ in range(20)])
part_numbers = np.arange(100, 1000)
gen.shuffle(part_numbers)
prod = pd.DataFrame(test_data, columns=["Quantity", "LotSize", "DueTime", "TimeForLot", "ImportanceFactor"], index=[f'Part--{part_numbers[i]}' for i in range(len(test_data))])
print(prod)

           Quantity  LotSize  DueTime  TimeForLot  ImportanceFactor
Part--411      15.0     15.0     17.0        89.0          0.917401
Part--370      11.0     11.0      4.0       151.0          4.527931
Part--322      12.0      6.0      1.0       106.0          4.293599
Part--551      10.0      9.0     15.0        68.0          3.518308
Part--532      16.0      5.0     22.0        87.0          1.658503
Part--708       3.0      6.0      7.0        67.0          4.895185
Part--201       6.0     14.0     13.0       169.0          2.318881
Part--170      17.0     12.0     21.0        58.0          1.754301
Part--791       5.0      5.0     24.0        89.0          0.475970
Part--278      18.0      5.0     23.0       186.0          1.653415
Part--909      13.0      7.0     17.0        87.0          0.753457
Part--812      12.0      7.0     22.0        65.0          1.396935
Part--463      12.0     10.0     12.0       171.0          3.381329
Part--688      12.0     14.0      1.0       150.

In [ ]:
js = prod_to_jobs(prod)
jobs_df = pd.DataFrame(js, index=["OpName", "LotSize", "TimeforLot", "DueTime", "Imp.Factor", "AssignedShifts"]).transpose()
jobs_df.iloc[-10:, :]

,OpName,LotSize,TimeforLot,DueTime,Imp.Factor,AssignedShifts
21,Part--463_1,10,171,12,3.381329,21
22,Part--553_0,3,87,3,1.078261,22
23,Part--553_1,3,87,3,1.078261,23
24,Part--553_2,3,87,3,1.078261,24
25,Part--553_3,3,87,3,1.078261,25
26,Part--514_0,8,96,5,0.92974,26
27,Part--514_1,8,96,5,0.92974,27
28,Part--955_0,4,111,8,2.089392,28
29,Part--955_1,4,111,8,2.089392,29
30,Part--955_2,4,111,8,2.089392,30


In [ ]:
number_of_jobs = len(jobs_df.OpName)
number_of_shifts = np.max(jobs_df.AssignedShifts) + 20
number_of_machines = 4

importance_factors = jobs_df["Imp.Factor"].values
print(importance_factors.shape, importance_factors.min())
due_dates = 5 + jobs_df["DueTime"].values
proc_time = np.ones((number_of_jobs, number_of_machines,), dtype=int)
available_machines = gen.choice([2, 3], size=(number_of_shifts, number_of_machines,), replace=True) * np.ones((number_of_shifts, number_of_machines,), dtype=int)

Jobs2Plan_model = build_concr_model(number_of_jobs, number_of_shifts, number_of_machines,
                                    tuple(np.arange(1, number_of_jobs + 1, dtype=int)),
                                    {i + 1 : importance_factors[i] for i in range(importance_factors.shape[0])},
                                    {i + 1 : due_dates[i] for i in range(due_dates.shape[0])},
                                    {(i + 1, j + 1) : proc_time[i, j] for i in range(proc_time.shape[0]) for j in range(proc_time.shape[1])},
                                    {(i + 1, j + 1) : available_machines[i, j] for i in range(available_machines.shape[0]) for j in range(available_machines.shape[1])}
                                    )

(31,) 0.4759700215071738


In [ ]:
job_plan_results = opt.solve(Jobs2Plan_model, tee=True, load_solutions=False)

Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms
MIP  has 448 rows; 6200 cols; 23250 nonzeros; 6200 integer variables (6200 binary)
Coefficient ranges:
  Matrix [1e+00, 5e+01]
  Cost   [5e-01, 2e+02]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 3e+01]
Presolving model
426 rows, 5109 cols, 18885 nonzeros  0s
426 rows, 4276 cols, 15605 nonzeros  0s
418 rows, 2670 cols, 9390 nonzeros  0s
390 rows, 1968 cols, 7157 nonzeros  0s
346 rows, 1968 cols, 7155 nonzeros  0s

Solving MIP model with:
   346 rows
   1968 cols (1968 binary, 0 integer, 0 implied int., 0 continuous, 0 domain fixed)
   7155 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; J => Feasibility jump;
     H => Heuristic; L => Sub-MIP; P => Empty MIP; R => Randomized rounding; Z => ZI Round;
     I => Shifting; S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

